# Курс Python. Занятие 4. Классы, магические методы и декораторы

## Содержание

[Часть 1. Ещё раз про функции](#Часть-1.-Ещё-раз-про-функции)

* [Вспомним про: функции, области видимости, LEGB](#Вспомним-про:-функции,-области-видимости,-LEGB
)
* [Замыкания: Closures](#Замыкания:-Closures
)
* [Декораторы](#Декораторы
)

[Часть 2. Классы](#Часть-2.-Классы)

* [Общий синтаксис](#Общий-синтаксис
)
* [Атрибуты, методы и self](#Атрибуты,-методы-и-self)
* [Инкапсуляция в Питоне](#Инкапсуляция-в-Питоне)
* [Наследование](#Наследование)
* [Магические методы](#Магические-методы)
* [Классы в качестве структур данных](#Классы-в-качестве-структур-данных)
* [Итераторы и генераторы](#Итераторы-и-генераторы)

[Часть 3. Исключения](#Часть-3.-Исключения)

* [Синтаксис: try - except - else - finally](#Синтаксис:-try---except---else---finally)
* [Ключевое слово raise](#Ключевое-слово-raise
)
* [Объявление новых типов исключений](#Объявление-новых-типов-исключений)

## Часть 1. Ещё раз про функции

### Вспомним про: функции, области видимости, LEGB
- Функции в Питоне - полноценные объекты, их можно присвоить переменным
- У них есть атрибуты, методы, их можно передавать в качестве аргументов и возвращать из других функций
- Функции можно объявлять внутри других функций (и вообще в любых местах кода)
- С каждой функцией связана связана своя локальная область видимости
- 4 области видимости: local, enclosing, global, builtin

In [41]:
# Пример областей видимости (из документации Питона)

def scope_test():
    spam = "test spam"
    
    def do_local():
        spam = "local spam"
        
    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"
        
    def do_global():
        global spam
        spam = "global spam"
  
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


In [42]:
# Подробнее про enclosing область видимости

def add_number(a):
    x = 2
    def add_some():
        print("x =", x)
        return a + x
    return add_some()

print("x =", add_number(5))

x = 2
x = 7


### Замыкания: Closures
Замыкание (closure) в программировании — это функция, в теле которой присутствуют ссылки на переменные, объявленные вне тела этой функции в окружающем коде и не являющиеся ее параметрами.

In [3]:
# Определим функцию умножения двух чисел

def multiply(a, b):
    return a * b

multiply(5, 2)

10

In [4]:
# Определим функцию умножения числа на 5

def multiply_by_5(a):
    return multiply(5, a)

multiply_by_5(2)

10

In [5]:
# Определим функцию, которая возвращает функцию, умножающую число на число

def multiply(a):
    def helper(b):
        print(a)        # Печатаем enclosing переменную a
        return a * b
    return helper

multiply(5)(2)

5


10

In [6]:
# С помощью новой функции создадим функцию умножения на 5

multiply_by_5 = multiply(5)

print(multiply_by_5)
print(multiply_by_5(2))             # Переменная a со значением 5 осталась, поскольку на неё ссылается helper

<function multiply.<locals>.helper at 0x00000211974E2400>
5
10


### Декораторы
Декораторы — это, по сути, просто своеобразные «обёртки», которые дают нам возможность делать что-либо до и после того, что сделает декорируемая функция, не изменяя её.

In [7]:
# Допустим у нас есть обычная функция

def simple_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?..")

# И мы хотим вызвать код до и после её выполнения

def decorator(function_to_decorate):
    
    def wrapper_around_function_to_decorate():

        print("Я - код, который отработает до вызова функции")
        
        function_to_decorate()

        print("А я - код, срабатывающий после")
        
    return wrapper_around_function_to_decorate

simple_function()
simple_function_decorated = decorator(simple_function)
simple_function_decorated()

Я простая одинокая функция, ты ведь не посмеешь меня изменять?..
Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?..
А я - код, срабатывающий после


In [8]:
# Перезапишем имя обычной функции на вызов декоратора для удобства
# Теперь обычной функции в старом виде нет

simple_function = decorator(simple_function)
simple_function()

Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?..
А я - код, срабатывающий после


Спецсимвол @ - удобный синтаксис для декорирования функций прямо во время объявления

In [9]:
@decorator
def simple_function():
    print("Оставь меня в покое")

simple_function()

Я - код, который отработает до вызова функции
Оставь меня в покое
А я - код, срабатывающий после


In [10]:
# Функции можно декорировать несколькими декораторами

@decorator
@decorator
def simple_function():
    print("Оставь меня в покое")

simple_function()

Я - код, который отработает до вызова функции
Я - код, который отработает до вызова функции
Оставь меня в покое
А я - код, срабатывающий после
А я - код, срабатывающий после


In [11]:
# Декоратор общего назначения для функций с параметрами

def decorator_general(function_to_decorate):
    
    def wrapper_around_function_to_decorate(*args, **kwargs):

        print("args:", args)
        print("kwargs:", kwargs)
        function_to_decorate(*args, **kwargs)       
        
    return wrapper_around_function_to_decorate

# Функция с параметрами задекорированная нашим декоратором

def function_with_params(*args, **kwargs):
    print("my args:", args)
    print("my kwargs:", kwargs)

function_with_params = decorator_general(function_with_params)
function_with_params(1, 2, 3, n=10, word="hello")

args: (1, 2, 3)
kwargs: {'n': 10, 'word': 'hello'}
my args: (1, 2, 3)
my kwargs: {'n': 10, 'word': 'hello'}


In [12]:
@decorator_general
def function_with_params(*args, **kwargs):
    print("my args:", args)
    print("my kwargs:", kwargs)

function_with_params(1, 2, 3, n=10, word="hello")

args: (1, 2, 3)
kwargs: {'n': 10, 'word': 'hello'}
my args: (1, 2, 3)
my kwar|gs: {'n': 10, 'word': 'hello'}


Есть ещё декораторы с параметрами, но это сложно, так что потом или самостоятельно.

## Часть 2. Классы

Объектно-ориентированное программирование (ООП) — парадигма программирования, в которой основными концепциями являются понятия объектов и классов.Использование классов дает преимущества абстрактного подхода в программировании.

Основные свойства классов:
* **Абстрагирование**: выделить важное, обычно абстракция данных
* **Инкапсуляция**: у классов есть данные и методы работы с ними, не все могут быть публичными
* **Наследование**: на основе одних классов можно создавать другие, повторяющих и расширяющих их функционал
* **Полиморфизм**: разное поведение одного и того же метода в разных классах
* **Композиция**: классы могут содержать другие классы, чтобы расширить свой функционал

Объектно-ориентированный подход в программировании подразумевает следующий алгоритм действий:

1. Описывается **проблема** с помощью обычного языка с использованием понятий, действий, прилагательных.
2. На основе **понятий** формулируются **классы**.
3. На основе **действий** проектируются **методы**.
4. **Реализуются** методы и атрибуты.

Наиболее важные особенности классов в Питоне:
* множественное наследование
* производный класс может переопределить любые методы базовых классов
* в любом месте можно вызвать метод с тем же именем базового класса
* все атрибуты класса в питоне по умолчанию являются public, т.е. доступны отовсюду
* все методы — виртуальные, т.е. перегружают базовые

### Общий синтаксис
В общем случае класс - это составной оператор, кусок кода, как if, for, def. <br><br>
Одно отличие: при объявлении класса с помощью ключевого слова `class` создаётся объект Класс (как объект Функция).

In [13]:
# Синтаксис

class MyClass:
    pass

print(MyClass)

<class '__main__.MyClass'>


Экземпляр класса создаётся при "вызове объекта Класс" и присваивании результата переменной. <br><br>

Этот экземпляр ещё называют объектом или инстансом (instance, instantiation).

In [14]:
my_class = MyClass()
print(my_class) 

### Атрибуты, методы и self

С точки зрения Питона атрибуты и методы класса - одно и тоже, поскольку всё - объекты.<br><br>
Создание экзэмпляра класса - по сути инициализация, то есть создание атрибутов и задание им значений.<br><br>
Для этого используется специальная функция `__init__` и идентификатор инстанса `self` (аналог this в С++).

In [40]:
# Пример объявления (создания) класса, имеющего атрибуты и методы класса и атрибуты и методы инстанса

class MyClass:

    class_attr = 0

    def __init__(self, attr):
        self.instance_attr = attr
    
    def class_method():
        print("I'm a class method")

    def instance_method(self):
        print("I'm an instance method")
    
instance1 = MyClass(1)
instance2 = MyClass(2)

print("Class attr:", instance1.class_attr, instance2.class_attr)
print("Instance attr:", instance1.instance_attr, instance2.instance_attr)

MyClass.class_method()
instance1.instance_method()
instance2.instance_method()

Class attr: 0 0
Instance attr: 1 2
I'm a class method
I'm an instance method
I'm an instance method


In [16]:
# Но ошибка при вызове метода класса у его экзэмпляра

instance1.class_method()

TypeError: class_method() takes 0 positional arguments but 1 was given

Что произошло? <br><br>Когда мы вызываем метод у экзэмпляра, он автоматически передаёт ссылку на себя в этот метод первым аргументом. <br><br>
Поэтому мы пишем `self` первым параметром при объявлении методов экзэмпляров класса.<br><br>
Такие методы ещё называют bound-методами.

In [17]:
# Чтобы объявить статический метод (метод класса, который можно вызывать у каждого экзэмпляра), 
# используют декоратор @staticmethod

class MyClass:

    class_attr = 0

    def __init__(self, attr):
        self.instance_attr = attr
        
    @staticmethod
    def class_method():
        print("I'm a class method")

    def instance_method(self):
        print("I'm an instance method")
        
instance1 = MyClass(1)
instance2 = MyClass(2)
        
MyClass.class_method()
instance1.class_method()
instance2.class_method()    

I'm a class method
I'm a class method
I'm a class method


Атрибуты класса к его экзэмпляру можно добавлять на лету

In [18]:
instance1.my_attr = 'hello'

print(instance1.my_attr)

hello


### Инкапсуляция в Питоне

Вообще в Питоне НЕ принято что-то прятать от программиста, но если очень нужно отделить внутреннюю логику класса (protected и private) и его внешний интерфейс (public), то пользуются следующими договорённостями:
- атрибуты (свойства и методы), которые нужно скрыть, называются с символа нижнее подчёркивание "_" (underscore)
- атрибуты, которые совсем приватные, начинаются с двух нижних подчёркиваний "__" (double underscore, dunder)

In [37]:
class Incapsulation:
    
    def __init__(self):
        self.public = 'public'
        self._protected = 'protected'
        self.__private = 'private'
    
    def public_method(self):
        print("I'm public")
    
    def _protected_method(self):
        print("I'm protected")
        
    def __private_method(self):
        print("I'm private")

instance = Incapsulation()

# Посмотрим переменные экзэмпляра
instance.__dict__

{'public': 'public',
 '_protected': 'protected',
 '_Incapsulation__private': 'private'}

In [39]:
# Вызовем методы

instance.public_method()
instance._protected_method()
instance._Incapsulation__private_method()

I'm public
I'm protected
I'm private


### Наследование


In [43]:
# Класс просто человек с именем и без работы по умолчанию :(

class Person:
    
    def __init__(self, name, job=None, salary=0):
        self.name = name
        self.job = job
        self.salary = salary
        
    def print_person(self):
        print(f'Person. Name = {self.name}, Job = {self.job}, Salary = {self.salary}')

# Класс менеджер, функция super() используется для получения родительского класса

class Manager(Person):
    
    def __init__(self, name, salary):
        super().__init__(name, 'Manager', salary)        

In [20]:
person = Person('John')
manager = Manager('Mary', 1000)

person.print_person()
manager.print_person()

Person. Name = John, Job = None, Salary = 0
Person. Name = Mary, Job = Manager, Salary = 1000


Возможно также множественное наследование, но лучше не стоит :)

### Магические методы

Метод `__init__` - один из множества предопределённых имён методов, которые делают разное. Их называют магическими методами.<br><br>
Перечислим основные группы магических методов:
- Сравнения
- Математические
- Конвертации типов
- Строкового представления
- Работы с атрибутами
- Работы с последовательностями
- Работы с контекстными менеджерами
- Вызова

Все методы перечислять долго, есть руководства, например, [ЭТО](https://habr.com/post/186608/).<br>
Приведём пример.

`__str__` и `__repr__` - для строкового представления экземпляров класса

In [21]:
class Person:
    
    def __init__(self, name, job=None, salary=0):
        self.name = name
        self.job = job
        self.salary = salary
        
    # Представление для программиста (по сути строчка кода с конструктором)
    
    def __repr__(self):
        return f'Person(name={self.name}, job={self.job}, salary={self.salary})'

    # Представление для пользователя (при выводе на печать)
    
    def __str__(self):
        return f'Person: Name = {self.name}, Job = {self.job}, Salary = {self.salary}'
    
person = Person('John', 'Manager', 2000)

print(person)
person

Person: Name = John, Job = Manager, Salary = 2000


Person(name=John, job=Manager, salary=2000)

### Классы в качестве структур данных

Поскольку классы позволяют добавлять новые атрибуты к его экзэмплярам на лету, это можно использовать для создания структур данных из классов.

In [22]:
class Person:
    pass

person = Person()
person.name = 'John'
person.job = 'Manager'

print(person.name, person.job)

John Manager


Но в Питоне с версии 3.7 для этих целей ввели Dataclasses - классы данных. Они позволяют явно объявить классы как структуры данных.<br><br>
Для этого нужно сделать импорт `from dataclasses import dataclass` и обернуть создаваемый декоратором @dataclass и перечислить в теле класса атрибуты и типы со значениями по умолчанию, а также методы. Декоратор сам создаст методы `__init__`, `__repr__`, `__eq__`.<br><br>
Подробнее про датаклассы можно почитать на [хабре](https://habr.com/post/415829/).

In [23]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    '''Класс данных для системы инвентаря'''
    
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

ModuleNotFoundError: No module named 'dataclasses'

### Итераторы и генераторы

Итераторы - классы, реализующие магические методы `__iter__` и `__next__`. <br><br>
Генераторы - функции, возвращающие результат с помощью `yield`.<br><br>
И те, и те - возвращают по одному "элементу" за "раз" и используются в циклах или явно с функцией `iter()`.

In [24]:
# Итератор - класс

class Reverse:
    """Итератор для обхода переданной ему последовательности в обратном порядке"""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    # Всегда возвращает экзэмпляр класса
    
    def __iter__(self):
        return self

    # Возвращает следующий "элемент" или вызывает исключение StopIteration
    
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]
    
rev = list(Reverse('Hello, World!'))
print(''.join(rev))

!dlroW ,olleH


In [25]:
# Генератор - функция. Генератор проще :)

def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]

rev = list(reverse('Hello, World!'))
print(''.join(rev))

!dlroW ,olleH


## Часть 3. Исключения

Исключения (exceptions) - ещё один тип данных в Python. Исключения необходимы для того, чтобы сообщать программисту об ошибках.<br><br> Исключения можно ловить, обрабатывать, вызывать.<br><br>
Исключения - иерархия классов, которые наследуются друг от друга и у которых есть соответствующие названия. Часть иерархии:
```
BaseException
 +-- SystemExit
 +-- KeyboardInterrupt
 +-- GeneratorExit
 +-- Exception
      +-- StopIteration
      +-- StopAsyncIteration
      +-- ArithmeticError
      |    +-- FloatingPointError
      |    +-- OverflowError
      |    +-- ZeroDivisionError
      +-- AssertionError
      +-- AttributeError
      +-- BufferError
      +-- EOFError
      +-- ImportError
           +-- ModuleNotFoundError
................
```

### Синтаксис: try - except - else - finally

In [26]:
# Блок кода, в котором может возникнуть исключение

# Сначала пытаемся сделать что-то опасное

try:  
    a = 10/0
    print (a)

# Потом перехватываем конкретную ошибку (или ошибки через запятую)
    
except ZeroDivisionError as e:  
    print("Zero division exception raised." , "Text of exception:", e)

# Несколько except работает как elif, обычно самые общие исключения - внизу

except Exception:
    print("Other type of exception raised.")
    
# Если всё прошло хорошо, то выполнить код в else

else:  
    print("Do this if no exceptions")
    
# Код в finally выполнить в любом случае
finally:
    print("Do this everytime")

Zero division exception raised. Text of exception: division by zero
Do this everytime


### Ключевое слово raise
Исключения можно порождать явно. Для этого используется ключевое слово raise и имя класса исключения.

In [27]:
try:  
    a = 10/2
    print (a)
    
    # Явно вызываем (бросаем) исключение нужного класса, текст передаём как параметр
    
    raise ZeroDivisionError('Всё равно деление на ноль :)')

except ZeroDivisionError as e:  
    print("Zero division exception raised." , "Text of exception:", e)

5.0
Zero division exception raised. Text of exception: Всё равно деление на ноль :)


### Объявление новых типов исключений

Поскольку исключения - классы, от них можно наследоваться (обычно от Exception).

In [28]:
# Самый простой вариант

class MyException(Exception):
    pass

try:
    raise MyException("My Exception")
except MyException as e:
    print(e)

My Exception
